# Part 2 of Segmenting And Clustering Neighborhoods in Toronto, Canada

### Part 1

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')
Canada_data = Canada_data.find('table')

In [3]:
entries = Canada_data.findAll('td')
table = []
for i, entry in enumerate(entries): 
    line = re.split(r'<[^>]*>',str(entry))
    row = []
    for w in line:
        w = w.replace('  ','').replace('(','').replace(')','')
        if '/' in w:
            extras = w.split('/')
            for e in extras:
                if len(e) > 2:
                    row.append(e.strip())
        elif len(w) > 2 and w!= ' / ':
            row.append(w.strip())
    table.append(row)
table[:5]

[['M1A', 'Not assigned'],
 ['M2A', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park', 'Harbourfront']]

In [26]:
First = []
Second = []
Third = []
for row in table:
    if len(row) > 2:
        First.append(row[0])
        Second.append(row[1])
        s = ""
        for w in row[2:]:
            s+= w + ", "
        Third.append(s)
        
            
            
        
df = pd.DataFrame(list(zip(First, Second, Third)), 
               columns =['Postal Code', 'Borough', 'Neigborhood'])
df

,Postal Code,Borough,Neigborhood
0,M3A,North York,"Parkwoods,"
1,M4A,North York,"Victoria Village,"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront,"
3,M6A,North York,"Lawrence Manor, Lawrence Heights,"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government,"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill, North,"
99,M4Y,Downtown Toronto,"Church and Wellesley,"
100,M7Y,East Toronto,"Business reply mail, Processing Centre, 969 Ea..."
101,M8Y,Etobicoke,"Old Mill, South, King's Mill Park, Sunnylea, H..."


In [27]:
df.shape

(103, 3)

### Part 2

In [28]:
geo_data = pd.read_csv('Geospatial_Coordinates.csv')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
df = pd.merge(df, geo_data, on='Postal Code')
df.head(12)

,Postal Code,Borough,Neigborhood,Latitude,Longitude
0,M3A,North York,"Parkwoods,",43.753259,-79.329656
1,M4A,North York,"Victoria Village,",43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront,",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights,",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government,",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue,",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge,",43.806686,-79.194353
7,M3B,North York,"Don Mills, North,",43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens,",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson,",43.657162,-79.378937


In [30]:
df.shape

(103, 5)